# Deploy the Model

The pipeline that was executed created a Model Package version within the specified Model Package Group. Of particular note, the registration of the model/creation of the Model Package was done so with approval status as `PendingManualApproval`.

As part of SageMaker Pipelines, data scientists can register the model with approved/pending manual approval as part of the CI/CD workflow.

We can also approve the model using the SageMaker Studio UI or programmatically as shown below.

In [ ]:
from botocore.exceptions import ClientError

import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

import botocore.config

config = botocore.config.Config(
    user_agent_extra='dsoaws/1.0'
)

sm = boto3.Session().client(service_name="sagemaker", 
                            region_name=region,
                            config=config)

# List Pipeline Execution Steps


In [ ]:
%store -r pipeline_name

In [ ]:
print(pipeline_name)

In [ ]:
%%time

import time
from pprint import pprint

executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

while pipeline_execution_status == "Executing":
    try:
        executions_response = sm.list_pipeline_executions(PipelineName=pipeline_name)["PipelineExecutionSummaries"]
        pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
    except Exception as e:
        print("Please wait...")
        time.sleep(30)

pprint(executions_response)

In [ ]:
pipeline_execution_status = executions_response[0]["PipelineExecutionStatus"]
print(pipeline_execution_status)

In [ ]:
pipeline_execution_arn = executions_response[0]["PipelineExecutionArn"]
print(pipeline_execution_arn)

In [ ]:
from pprint import pprint

steps = sm.list_pipeline_execution_steps(PipelineExecutionArn=pipeline_execution_arn)

pprint(steps)

# View Registered Model

In [ ]:
for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "RegisterModel":
        model_package_arn = execution_step["Metadata"]["RegisterModel"]["Arn"]
        break
print(model_package_arn)

In [ ]:
model_package_update_response = sm.update_model_package(
    ModelPackageArn=model_package_arn,
    ModelApprovalStatus="Approved",  # Other options are Rejected and PendingManualApproval
)

# View Created Model

In [ ]:
for execution_step in steps["PipelineExecutionSteps"]:
    if execution_step["StepName"] == "CreateModel":
        model_arn = execution_step["Metadata"]["Model"]["Arn"]
        break
print(model_arn)

pipeline_model_name = model_arn.split("/")[-1]
print(pipeline_model_name)

# Create Model Endpoint from Model Registry
More details here:  https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-deploy.html


In [ ]:
import time

timestamp = int(time.time())

model_from_registry_name = "bert-model-from-registry-{}".format(timestamp)
print("Model from registry name : {}".format(model_from_registry_name))

model_registry_package_container = {
    "ModelPackageName": model_package_arn,
}

In [ ]:
from pprint import pprint

create_model_from_registry_response = sm.create_model(
    ModelName=model_from_registry_name, ExecutionRoleArn=role, PrimaryContainer=model_registry_package_container
)
pprint(create_model_from_registry_response)

In [ ]:
model_from_registry_arn = create_model_from_registry_response["ModelArn"]
model_from_registry_arn

In [ ]:
endpoint_config_name = "bert-model-from-registry-epc-{}".format(timestamp)
print(endpoint_config_name)

create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m5.4xlarge",
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": pipeline_model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

In [ ]:
# Pick up the existing `pipeline_endpoint_name` if it was already created

%store -r pipeline_endpoint_name

In [ ]:
try:
    print("Using existing Pipeline EndpointName: {}".format(pipeline_endpoint_name))
except NameError:
    timestamp = int(time.time())
    pipeline_endpoint_name = "bert-model-from-registry-ep-{}".format(timestamp)
    print("Created Pipeline EndpointName={}".format(pipeline_endpoint_name))

    create_endpoint_response = sm.create_endpoint(
        EndpointName=pipeline_endpoint_name, EndpointConfigName=endpoint_config_name
    )
    print(create_endpoint_response["EndpointArn"])

In [ ]:
%store pipeline_endpoint_name

In [ ]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(
            region, pipeline_endpoint_name
        )
    )
)

# _Wait Until the Endpoint is Deployed_
_Note:  This will take a few minutes.  Please be patient._

In [ ]:
%%time

waiter = sm.get_waiter("endpoint_in_service")
waiter.wait(EndpointName=pipeline_endpoint_name)

# _Wait Until the Endpoint ^^ Above ^^ is Deployed_

# Predict the star_rating with Ad Hoc review_body Samples

In [ ]:
import json
from sagemaker.tensorflow.model import TensorFlowPredictor
from sagemaker.serializers import JSONLinesSerializer
from sagemaker.deserializers import JSONLinesDeserializer

predictor = TensorFlowPredictor(
    endpoint_name=pipeline_endpoint_name,
    sagemaker_session=sess,
    model_name="saved_model",
    model_version=0,
    accept_type="application/jsonlines",
    serializer=JSONLinesSerializer(),
    deserializer=JSONLinesDeserializer(),
)

# These predictions are not that great, we know.  We need to train longer!

In [ ]:
inputs = [{"features": ["This is great!"]}, {"features": ["This is bad."]}]

predicted_classes = predictor.predict(inputs)

for predicted_class in predicted_classes:
    print("Predicted class: {}".format(predicted_class))

# Release Resources

In [ ]:
# sm.delete_endpoint(
#      EndpointName=pipeline_endpoint_name
# )

In [ ]:
# %%html

# <p><b>Shutting down your kernel for this notebook to release resources.</b></p>
# <button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>

# <script>
# try {
#     els = document.getElementsByClassName("sm-command-button");
#     els[0].click();
# }
# catch(err) {
#     // NoOp
# }
# </script>